<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/sir_rec_disc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# -----------------------------
# Modelo SIR discreto
# -----------------------------
def sir_disc(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0, T):
    passos = int(T) + 1
    S, I, R, N = np.zeros(passos), np.zeros(passos), np.zeros(passos), np.zeros(passos)
    t = np.arange(0, T + 1)
    S[0] = max(0, N0 - I0 - R0)
    I[0], R[0] = I0, R0
    N[0] = S[0] + I[0] + R[0]
    max_threshold = 1e8

    for k in range(passos - 1):
        N[k] = S[k] + I[k] + R[k]
        if N[k] > max_threshold:
            S[k+1:] = I[k+1:] = R[k+1:] = N[k+1:] = np.nan
            break
        S[k+1] = S[k] + (Lambda + nu * N[k] - beta * S[k] * I[k] / N[k] - mu * S[k])
        I[k+1] = I[k] + (beta * S[k] * I[k] / N[k] - gamma * I[k] - (mu + alpha) * I[k])
        R[k+1] = R[k] + (gamma * I[k] - mu * R[k])
        N[k+1] = S[k+1] + I[k+1] + R[k+1]
    return t, S, I, R, N

# -----------------------------
# Gráfico interativo
# -----------------------------
def plot_sir(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0_val, T):
    t, S, I, R, N = sir_disc(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0_val, T)
    I_eq = I[~np.isnan(I)][-1]
    N_eq = N[~np.isnan(N)][-1]

    fig, axs = plt.subplots(2, 2, figsize=(14, 10), dpi=100)

    axs[0, 0].plot(t, I, color='blue')
    axs[0, 0].set_xlabel('$t$ (dias)')
    axs[0, 0].set_ylabel('$I(t)$')
    axs[0, 0].grid(True)
    axs[0, 0].text(0.7 * T, 0.9 * np.nanmax(I), f'$I^* = {I_eq:.2f}$', fontsize=10,
                   bbox=dict(facecolor='white', alpha=0.7))

    axs[0, 1].plot(t, N, color='darkgreen')
    axs[0, 1].set_xlabel('$t$ (dias)')
    axs[0, 1].set_ylabel('$N(t)$')
    axs[0, 1].grid(True)
    axs[0, 1].text(0.7 * T, 0.9 * np.nanmax(N), f'$N^* = {N_eq:.2f}$', fontsize=10,
                   bbox=dict(facecolor='white', alpha=0.7))

    axs[1, 0].plot(N, I, color='purple')
    axs[1, 0].plot(N_eq, I_eq, 'ro')
    axs[1, 0].set_xlabel('$N(t)$')
    axs[1, 0].set_ylabel('$I(t)$')
    axs[1, 0].set_title('Diagrama de fase')
    axs[1, 0].grid(True)

    axs[1, 1].axis('off')
    plt.tight_layout()
    plt.show()

# -----------------------------
# Sliders
# -----------------------------
beta_slider = widgets.FloatSlider(value=0.3, min=0.01, max=1.5, step=0.001, description='beta',readout_format='.4f')
gamma_slider = widgets.FloatSlider(value=0.122, min=0.001, max=1.0, step=0.001, description='gamma',readout_format='.4f')
nu_slider = widgets.FloatSlider(value=0.000, min=0.0, max=0.1, step=0.0001, description='nu',readout_format='.4f')
mu_slider = widgets.FloatSlider(value=0.0072, min=0.0001, max=0.1, step=0.0001, description='mu',readout_format='.4f')
alpha_slider = widgets.FloatSlider(value=0.042, min=0.001, max=0.1, step=0.0001, description='alpha',readout_format='.4f')
Lambda_slider = widgets.FloatSlider(value=13, min=0, max=100, step=1, description='Lambda',readout_format='.4f')
N0_slider = widgets.IntSlider(value=10000, min=100, max=100000, step=10, description='N0')
I0_slider = widgets.IntSlider(value=10, min=0, max=500, step=1, description='I0')
R0_slider = widgets.FloatSlider(value=2.5, min=0.5, max=10, step=0.01, description='R0',readout_format='.4f')
T_slider = widgets.IntSlider(value=1000, min=50, max=5000, step=10, description='T')

ui = widgets.VBox([
    beta_slider, gamma_slider, nu_slider, mu_slider, alpha_slider,
    Lambda_slider, N0_slider, I0_slider, R0_slider, T_slider
])

out = widgets.interactive_output(plot_sir, {
    'beta': beta_slider, 'gamma': gamma_slider, 'nu': nu_slider,
    'mu': mu_slider, 'alpha': alpha_slider, 'Lambda': Lambda_slider,
    'N0': N0_slider, 'I0': I0_slider, 'R0_val': R0_slider, 'T': T_slider
})

display(ui, out)


Output()